In [ ]:
import mne 
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')


raw = mne.io.read_raw_eeglab(f'rEEG\sub-001\eeg\sub-001_task-Rest_eeg.set',
                            preload=True)
raw.info  # check channels, sampling rate, etc.
print(raw.ch_names)


# After visual inspection, mark bad channels

raw.info['bads'] = [raw.ch_names[45], raw.ch_names[4], raw.ch_names[47]]
bads_to_remove = [raw.ch_names[_] for _ in [45, 4, 47]]
raw.drop_channels(bads_to_remove)
raw.plot(scalings='auto', block=True, title='Raw Data with Bad Channels Marked')

montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)


from mne.preprocessing import find_bad_channels_lof
# Find bad channels using LOF
bads_lof = find_bad_channels_lof(raw, n_neighbors=25, threshold=1.5, picks='eeg')
raw.info['bads'].extend(bads_lof)

raw.interpolate_bads(reset_bads=True)

raw.set_eeg_reference('average')
raw.plot(scalings='auto', block=True, title='Rereferenced Data')

raw.filter(1., 45., fir_design='firwin')
raw.plot(scalings='auto', block=True, title='Band Pass Filtered Data')

raw.resample(250, npad="auto")
raw.plot(scalings='auto', block=True, title='Resampled Data')

raw.save(f'rEEG/sub-001/preICA_raw.fif', overwrite=True)
print(f'Saved pre-ICA data for subject 001')

Reading c:\Users\User\Documents\EEG_Project\rEEG\sub-001\eeg\sub-001_task-Rest_eeg.fdt


Reading 0 ... 140829  =      0.000 ...   281.658 secs...
['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'AF8', 'AF4', 'F2', 'FCz']


C:\Users\User\AppData\Local\Temp\ipykernel_36552\1408467142.py:7: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(f'rEEG\sub-001\eeg\sub-001_task-Rest_eeg.set',


Channels marked as bad:
none


NameError: name 'onset' is not defined

### Load Data

In [ ]:
raw = mne.io.read_raw_eeglab('rEEG\sub-001\eeg\sub-001_task-Rest_eeg.set',
                              preload=True)
raw.info  # check channels, sampling rate, etc.
print(raw.ch_names)


### Mark Bad Channels (dataset specified)

In [ ]:
# After visual inspection, mark bad channels

raw.info['bads'] = [raw.ch_names[45], raw.ch_names[4], raw.ch_names[47]]
raw.plot(scalings='auto', block=True, title='Raw Data with Bad Channels Marked')

### Set montage


In [ ]:
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)

### Autodetect additional bad channels

In [ ]:
from mne.preprocessing import find_bad_channels_lof
# Find bad channels using LOF
bads_lof = find_bad_channels_lof(raw, n_neighbors=13, threshold=1.5, picks='eeg')
raw.info['bads'].extend(bads_lof) # Add detected bads to raw.info['bads']

### Interpolate bad channels

In [ ]:
raw.interpolate_bads(reset_bads=True)

### Re-reference to common average

In [ ]:
raw.set_eeg_reference('average')
raw.plot(scalings='auto', block=True, title='Rereferenced Data')

### Band-pass filter (1-45 Hz)

In [ ]:
raw.filter(1., 45., fir_design='firwin')
raw.plot(scalings='auto', block=True, title='Band Pass Filtered Data')

### Downsample to 250 Hz

### Plot PSD and cleaned signal

In [ ]:
raw.resample(250, npad="auto")
raw.plot(scalings='auto', block=True, title='Resampled Data')

# SAVE DATA PER SUBJECT BEFORE ICA INTO THEIR FOLDER

In [ ]:
raw.save('rEEG/sub-001/preICA_raw.fif', overwrite=True)
